In [1]:
import numpy as np
import keras
import AirSimClient
import time
import drone_env

from RL_net import DQNClass
from RL_dataset import Dataset

D:\Users\ruihanz\Anaconda3\envs\PY3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
np.set_printoptions(precision=3, suppress=True)
test_model = None
#test_model = keras.models.load_model("finala.h5")
test_model = keras.models.load_model("finalb.h5")

In [4]:
settings = {}

settings["learning_rate"] = 0.003
settings["reward_decay"] = 0.9
settings["e_greedy"] = 0.99
settings["e_greedy_increasement"] = 0
settings["memory_length"] = 81920
settings["batch_size"] = 32
settings["epochs"] = 1
settings["replace_target_iter"] = 50
settings["model"] = test_model
settings["n_actions"] = 7
settings["n_features"] = 6

aim = AirSimClient.Vector3r(32,38,-4)
start = AirSimClient.Vector3r(0,0,-5)

In [5]:
RL = DQNClass(settings)
env = drone_env.drone_env_gridworld()
RL.model.summary()



current = 0
count = 0
learns = 0
epochs = 0
succ = 0
learn = 0
treward = []
rc = 0

Waiting for connection: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               700       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 707       
Total params: 31,707
Trainable params: 31,707
Non-trainable params: 0
_________________________________________________________________


In [8]:


s = env.reset()
while True:
    
    action = RL.choose_action(s)
    
    s_,r,done,info = env.step(action)
    
    env.render(extra1 = str(count),extra2 = "reward: "+str(int(r))+" Done: "+str(succ)+" Epochs: "+str(epochs))
      
    count += 1
    current += 1
    if info == "success":
        succ += 1
    if current>100:
        done = True
        info = "out of steps"
    
    rc += r
    counter = RL.add_data(s, action, r, s_)
    
    if done:
        current = 0
        epochs += 1
        RL.model.save("test.h5")
        treward.append(rc)
        print ("reset due to",info,"epoch reward:",rc)
        print ("--------------------------------reset---------------------------")
        rc = 0
        env.reset()
        
    if count > 1:
        RL.learn(times = 2)
        learn += 1
        
    s = s_

101 distance: 69 position: [ 2  0 -6] reward: 0 Done: 6 Epochs: 6
target net replaced
102 distance: 64 position: [ 7  0 -6] reward: 4 Done: 6 Epochs: 6
103 distance: 59 position: [12  0 -6] reward: 4 Done: 6 Epochs: 6
104 distance: 55 position: [15  2 -7] reward: 3 Done: 6 Epochs: 6
105 distance: 50 position: [15  7 -7] reward: 3 Done: 6 Epochs: 6
106 distance: 45 position: [15 12 -7] reward: 3 Done: 6 Epochs: 6
107 distance: 40 position: [15 17 -7] reward: 4 Done: 6 Epochs: 6
108 distance: 35 position: [15 22 -7] reward: 4 Done: 6 Epochs: 6
109 distance: 30 position: [15 27 -7] reward: 4 Done: 6 Epochs: 6
110 distance: 26 position: [17 30 -8] reward: 2 Done: 6 Epochs: 6
111 distance: 21 position: [22 30 -8] reward: 3 Done: 6 Epochs: 6
112 distance: 17 position: [25 32 -9] reward: 3 Done: 6 Epochs: 6
113 distance: 12 position: [25 36 -8] reward: 4 Done: 6 Epochs: 6
114 distance: 7 position: [27 37 -6] reward: 100 Done: 6 Epochs: 6
reset due to success epoch reward: 142.0282461521881
--

KeyboardInterrupt: 

In [ ]:
RL.model.predict(np.array([[0,-2, -11, 0,-4, 0]]))

In [ ]:
RL.model.predict(np.array([[0,30, -5, 0,0, 0]]))

In [ ]:
print (treward)

In [ ]:
del env
env = drone_env.drone_env_gridworld()